# 11강: ETF 성과 & 리스크 지표 계산

이 강의에서는 **7강(성과·위험 지표)**, **8강(레버리지/인버스 특성)**을
**ETF + 벤치마크 시계열**로 실제로 구현합니다.

## 이 강의의 목표

1. ETF + 벤치마크 일봉 시계열 수집
2. 수익률 계산 (기간별, YTD, 설정일 이후)
3. 리스크 지표 (변동성, MDD, Sharpe, Sortino)
4. 상대 지표 (Beta, Alpha, 추적오차, 상대수익률)
5. 드로다운 분석

## 입력 데이터

| 데이터 | 출처 | 용도 |
|--------|------|------|
| ETF 일봉 | 9강 저장 또는 신규 수집 | 수익률/리스크 계산 |
| 벤치마크 지수 일봉 | 신규 수집 | 상대 지표 계산 |

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import date, datetime, timedelta
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple

from kis_utils import (
    DATA_DIR,
    to_float,
    ensure_kis_token,
    call_kis_get,
)

## Step 0. 디렉터리 설정 및 벤치마크 매핑

In [2]:
# 디렉터리 설정
RAW_DATA_DIR = DATA_DIR / "raw"
DAILY_PRICE_DIR = DATA_DIR / "daily_prices"
RISK_SUMMARY_DIR = DATA_DIR / "etf_risk_summary"

DAILY_PRICE_DIR.mkdir(parents=True, exist_ok=True)
RISK_SUMMARY_DIR.mkdir(parents=True, exist_ok=True)

print(f"RAW_DATA_DIR    : {RAW_DATA_DIR}")
print(f"DAILY_PRICE_DIR : {DAILY_PRICE_DIR}")
print(f"RISK_SUMMARY_DIR: {RISK_SUMMARY_DIR}")

RAW_DATA_DIR    : /Users/ulift/workspace/py-etf-mango/script2/claude/data/raw
DAILY_PRICE_DIR : /Users/ulift/workspace/py-etf-mango/script2/claude/data/daily_prices
RISK_SUMMARY_DIR: /Users/ulift/workspace/py-etf-mango/script2/claude/data/etf_risk_summary


In [3]:
# 주요 ETF → 벤치마크 지수 매핑
ETF_BENCHMARK_MAP: Dict[str, str] = {
    # KOSPI 200 추종
    "069500": "2001",  # KODEX 200
    "102110": "2001",  # TIGER 200
    "148020": "2001",  # KBSTAR 200
    "152100": "2001",  # ARIRANG 200
    
    # KOSPI 200 레버리지/인버스
    "122630": "2001",  # KODEX 레버리지
    "252670": "2001",  # KODEX 200선물인버스2X
    "145670": "2001",  # KINDEX 인버스
    "252710": "2001",  # TIGER 200선물인버스2X
    
    # KOSPI
    "069660": "0001",  # KOSEF 200
    "091160": "0001",  # KODEX 종합주가
    
    # KOSDAQ 150
    "229200": "2203",  # KODEX KOSDAQ150
    "233740": "2203",  # TIGER KOSDAQ150
    "251340": "2203",  # KODEX KOSDAQ150레버리지
}

def get_benchmark_code(etf_code: str) -> str:
    """ETF 코드에 매핑된 벤치마크 지수 코드를 반환. 없으면 KOSPI(0001)."""
    return ETF_BENCHMARK_MAP.get(str(etf_code).zfill(6), "0001")

---

## Step 1. 일봉 시계열 수집

### 1-1. 국내주식 일봉 API (`FHKST03010100`)

In [4]:
def fetch_daily_price_once(
    code: str,
    start_date: str,
    end_date: str,
    *,
    period_div: str = "D",
    org_adj_prc: str = "0",
) -> pd.DataFrame:
    """
    국내주식 기간별시세 API를 '한 번' 호출하여 최대 100개 일봉 반환.
    """
    path = "/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"
    tr_id = "FHKST03010100"
    
    params = {
        "FID_COND_MRKT_DIV_CODE": "J",
        "FID_INPUT_ISCD": str(code).zfill(6),
        "FID_INPUT_DATE_1": start_date,
        "FID_INPUT_DATE_2": end_date,
        "FID_PERIOD_DIV_CODE": period_div,
        "FID_ORG_ADJ_PRC": org_adj_prc,
    }
    
    data = call_kis_get(path, tr_id, params)
    rows = data.get("output2") or []
    
    if not rows:
        return pd.DataFrame()
    
    df = pd.DataFrame(rows)
    df["code"] = str(code).zfill(6)
    df["date"] = pd.to_datetime(df["stck_bsop_date"], format="%Y%m%d")
    
    for col in ["stck_clpr", "stck_oprc", "stck_hgpr", "stck_lwpr", "acml_vol"]:
        if col in df.columns:
            df[col + "_float"] = df[col].apply(to_float)
    
    return df.sort_values("date").reset_index(drop=True)

### 1-2. 업종(지수) 일봉 API (`FHKUP03500100`)

In [5]:
def fetch_daily_index_price_once(
    index_code: str,
    start_date: str,
    end_date: str,
    *,
    period_div: str = "D",
) -> pd.DataFrame:
    """
    업종(지수) 기간별 시세 조회.
    FID_COND_MRKT_DIV_CODE = "U" (업종) 사용.
    """
    path = "/uapi/domestic-stock/v1/quotations/inquire-daily-indexchartprice"
    tr_id = "FHKUP03500100"
    
    params = {
        "FID_COND_MRKT_DIV_CODE": "U",
        "FID_INPUT_ISCD": str(index_code).zfill(4),
        "FID_INPUT_DATE_1": start_date,
        "FID_INPUT_DATE_2": end_date,
        "FID_PERIOD_DIV_CODE": period_div,
    }
    
    data = call_kis_get(path, tr_id, params)
    rows = data.get("output2") or []
    
    if not rows:
        return pd.DataFrame()
    
    df = pd.DataFrame(rows)
    df["code"] = str(index_code).zfill(4)
    df["date"] = pd.to_datetime(df["stck_bsop_date"], format="%Y%m%d")
    
    # 지수 종가를 'stck_clpr_float'로 통일 (호환성 유지)
    if "bstp_nmix_prpr" in df.columns:
        df["stck_clpr_float"] = df["bstp_nmix_prpr"].apply(to_float)
    
    return df.sort_values("date").reset_index(drop=True)

### 1-3. 긴 기간 데이터 수집 (청크 단위 반복)

In [6]:
def fetch_daily_price_range(
    code: str,
    start_date: str,
    end_date: str,
    *,
    chunk_size: int = 90,
) -> pd.DataFrame:
    """
    긴 기간의 일봉 데이터를 청크 단위로 수집.
    - 4자리 이하 코드: 지수 API
    - 6자리 코드: 주식/ETF API
    """
    raw_code = str(code).strip()
    is_index = len(raw_code) <= 4
    
    dt_start = datetime.strptime(start_date, "%Y%m%d").date()
    dt_end = datetime.strptime(end_date, "%Y%m%d").date()
    
    if dt_start > dt_end:
        raise ValueError(f"start_date({start_date}) > end_date({end_date})")
    
    frames = []
    cur_start = dt_start
    
    while cur_start <= dt_end:
        cur_end = min(cur_start + timedelta(days=chunk_size - 1), dt_end)
        
        s = cur_start.strftime("%Y%m%d")
        e = cur_end.strftime("%Y%m%d")
        
        print(f"  [fetch] {raw_code} ({'INDEX' if is_index else 'STOCK'}) {s} ~ {e}")
        
        if is_index:
            df_chunk = fetch_daily_index_price_once(raw_code, s, e)
        else:
            df_chunk = fetch_daily_price_once(raw_code, s, e)
        
        if not df_chunk.empty:
            frames.append(df_chunk)
        
        cur_start = cur_end + timedelta(days=1)
        time.sleep(0.1)
    
    if not frames:
        print(f"  ⚠️ 데이터 없음: {code}")
        return pd.DataFrame()
    
    df_all = pd.concat(frames, ignore_index=True)
    df_all = df_all.sort_values("date").drop_duplicates(subset=["code", "date"]).reset_index(drop=True)
    
    return df_all

---

## Step 2. 수익률 계산 (7강 연계)

In [7]:
def enrich_with_returns(df_price: pd.DataFrame) -> pd.DataFrame:
    """일봉 시세 DataFrame에 일간 수익률 컬럼을 추가."""
    df = df_price.copy()
    
    if "stck_clpr_float" not in df.columns:
        raise ValueError("stck_clpr_float 컬럼이 필요합니다.")
    
    df = df.sort_values("date").reset_index(drop=True)
    df["daily_ret"] = df["stck_clpr_float"].pct_change()
    
    return df


def calc_period_return(df_ret: pd.DataFrame, periods: int) -> Optional[float]:
    """최근 N 거래일에 대한 누적 수익률."""
    df = df_ret.dropna(subset=["daily_ret"]).copy()
    if len(df) < periods:
        return None
    
    last_slice = df.iloc[-periods:]
    cum = (1.0 + last_slice["daily_ret"]).prod() - 1.0
    return float(cum)


def calc_ytd_return(df_ret: pd.DataFrame) -> Optional[float]:
    """연초 이후(YTD) 수익률."""
    df = df_ret.dropna(subset=["daily_ret", "date"]).copy()
    if df.empty:
        return None
    
    current_year = df["date"].max().year
    ytd_data = df[df["date"].dt.year == current_year]
    
    if len(ytd_data) < 2:
        return None
    
    cum = (1.0 + ytd_data["daily_ret"]).prod() - 1.0
    return float(cum)


def calc_since_inception_return(
    df_ret: pd.DataFrame,
    inception_date: str
) -> Optional[float]:
    """설정일 이후 누적수익률."""
    df = df_ret.dropna(subset=["daily_ret", "date"]).copy()
    if df.empty:
        return None
    
    try:
        if "-" in inception_date:
            inception_dt = pd.to_datetime(inception_date)
        else:
            inception_dt = pd.to_datetime(inception_date, format="%Y%m%d")
    except Exception:
        return None
    
    since = df[df["date"] >= inception_dt]
    if len(since) < 2:
        return None
    
    cum = (1.0 + since["daily_ret"]).prod() - 1.0
    return float(cum)

---

## Step 3. 단일 자산 리스크 지표 (7강 연계)

In [8]:
def calc_volatility_annual(df_ret: pd.DataFrame) -> float:
    """연환산 변동성 = 일간 수익률 표준편차 × √252"""
    daily = df_ret["daily_ret"].dropna()
    if len(daily) < 2:
        return float("nan")
    return float(daily.std(ddof=1) * np.sqrt(252))


def calc_mdd(df_price: pd.DataFrame) -> float:
    """전체 기간 최대낙폭(MDD)."""
    price = df_price["stck_clpr_float"].dropna()
    if price.empty:
        return float("nan")
    cummax = price.cummax()
    dd = price / cummax - 1.0
    return float(dd.min())


def calc_mdd_period(df_price: pd.DataFrame, periods: int) -> float:
    """최근 N 거래일 기준 MDD."""
    df = df_price.dropna(subset=["stck_clpr_float"]).copy()
    if len(df) < periods:
        return float("nan")
    
    recent = df.tail(periods)
    price = recent["stck_clpr_float"]
    cummax = price.cummax()
    dd = price / cummax - 1.0
    return float(dd.min())


def calc_mdd_since_inception(
    df_price: pd.DataFrame,
    inception_date: str
) -> float:
    """설정일 이후 MDD."""
    df = df_price.dropna(subset=["stck_clpr_float", "date"]).copy()
    
    try:
        if "-" in inception_date:
            inception_dt = pd.to_datetime(inception_date)
        else:
            inception_dt = pd.to_datetime(inception_date, format="%Y%m%d")
    except Exception:
        return float("nan")
    
    since = df[df["date"] >= inception_dt]
    if since.empty:
        return float("nan")
    
    price = since["stck_clpr_float"]
    cummax = price.cummax()
    dd = price / cummax - 1.0
    return float(dd.min())


def calc_sharpe_sortino(
    df_ret: pd.DataFrame,
    *,
    risk_free_rate: float = 0.03,
) -> Tuple[float, float]:
    """Sharpe, Sortino 비율 계산."""
    daily = df_ret["daily_ret"].dropna()
    if len(daily) < 2:
        return float("nan"), float("nan")
    
    mean_daily = daily.mean()
    std_daily = daily.std(ddof=1)
    downside = daily[daily < 0]
    downside_std = downside.std(ddof=1) if len(downside) > 0 else float("nan")
    
    ann_ret = mean_daily * 252
    rf_ann = risk_free_rate
    
    # Sharpe
    if std_daily > 0:
        sharpe = (ann_ret - rf_ann) / (std_daily * np.sqrt(252))
    else:
        sharpe = float("nan")
    
    # Sortino
    if isinstance(downside_std, float) and downside_std > 0:
        sortino = (ann_ret - rf_ann) / (downside_std * np.sqrt(252))
    else:
        sortino = float("nan")
    
    return float(sharpe), float(sortino)

---

## Step 4. 상대 지표 (7강 연계)

In [9]:
def calc_beta_alpha(
    df_etf: pd.DataFrame,
    df_mkt: pd.DataFrame,
    *,
    risk_free_rate: float = 0.03,
) -> Tuple[float, float]:
    """ETF vs Market 일간 수익률 기반 Beta, Alpha 계산."""
    left = df_etf[["date", "daily_ret"]].rename(columns={"daily_ret": "ret_etf"})
    right = df_mkt[["date", "daily_ret"]].rename(columns={"daily_ret": "ret_mkt"})
    
    merged = pd.merge(left, right, on="date", how="inner").dropna()
    if len(merged) < 10:
        print("  ⚠️ Beta/Alpha 계산을 위한 공통 거래일이 너무 적습니다.")
        return float("nan"), float("nan")
    
    ret_etf = merged["ret_etf"]
    ret_mkt = merged["ret_mkt"]
    
    var_mkt = ret_mkt.var(ddof=1)
    if var_mkt <= 0:
        return float("nan"), float("nan")
    
    cov = np.cov(ret_etf, ret_mkt, ddof=1)[0, 1]
    beta = cov / var_mkt
    
    mean_etf_daily = ret_etf.mean()
    mean_mkt_daily = ret_mkt.mean()
    
    ann_ret_etf = mean_etf_daily * 252
    ann_ret_mkt = mean_mkt_daily * 252
    rf_ann = risk_free_rate
    
    capm_expected = rf_ann + beta * (ann_ret_mkt - rf_ann)
    alpha = ann_ret_etf - capm_expected
    
    return float(beta), float(alpha)


def calc_tracking_error(df_etf: pd.DataFrame, df_bm: pd.DataFrame) -> float:
    """추적오차(TE) = Std(Return_ETF - Return_BM) × √252"""
    left = df_etf[["date", "daily_ret"]].rename(columns={"daily_ret": "ret_etf"})
    right = df_bm[["date", "daily_ret"]].rename(columns={"daily_ret": "ret_mkt"})
    
    merged = pd.merge(left, right, on="date", how="inner").dropna()
    if len(merged) < 10:
        return float("nan")
    
    diff = merged["ret_etf"] - merged["ret_mkt"]
    te = diff.std(ddof=1) * np.sqrt(252)
    return float(te)


def calc_relative_return(
    df_etf: pd.DataFrame,
    df_bm: pd.DataFrame,
    periods: int
) -> Optional[float]:
    """ETF vs 벤치마크 상대수익률."""
    etf_ret = calc_period_return(df_etf, periods)
    bm_ret = calc_period_return(df_bm, periods)
    
    if etf_ret is None or bm_ret is None:
        return None
    
    return float(etf_ret - bm_ret)


def calc_avg_volume(df_etf: pd.DataFrame, days: int = 20) -> float:
    """최근 N일 평균 거래량."""
    if "acml_vol" not in df_etf.columns or df_etf.empty:
        return 0.0
    
    recent = df_etf["acml_vol"].tail(days)
    recent = pd.to_numeric(recent, errors='coerce').fillna(0)
    
    return float(recent.mean())

---

## Step 5. 드로다운 분석 (7강 연계)

In [10]:
def calc_drawdown_series(df_price: pd.DataFrame) -> pd.DataFrame:
    """드로다운 시계열 생성."""
    df = df_price[["date", "stck_clpr_float"]].dropna().copy()
    df = df.sort_values("date").reset_index(drop=True)
    
    df["cummax"] = df["stck_clpr_float"].cummax()
    df["drawdown"] = df["stck_clpr_float"] / df["cummax"] - 1
    
    return df


def calc_drawdown_stats(df_price: pd.DataFrame) -> Dict[str, Any]:
    """드로다운 상세 통계."""
    dd_series = calc_drawdown_series(df_price)
    
    if dd_series.empty:
        return {}
    
    dd = dd_series["drawdown"]
    
    return {
        "mdd": float(dd.min()),
        "avg_drawdown": float(dd.mean()),
        "drawdown_std": float(dd.std()),
        "time_underwater_pct": float((dd < 0).mean()),
        "current_drawdown": float(dd.iloc[-1]),
    }


def find_max_drawdown_period(df_price: pd.DataFrame) -> Dict[str, Any]:
    """MDD 발생 기간 찾기 (고점→저점→회복)."""
    dd_df = calc_drawdown_series(df_price)
    
    if dd_df.empty:
        return {}
    
    mdd_idx = dd_df["drawdown"].idxmin()
    mdd_date = dd_df.loc[mdd_idx, "date"]
    mdd_value = dd_df.loc[mdd_idx, "drawdown"]
    
    # 고점 찾기 (MDD 이전 최고점)
    before_mdd = dd_df.loc[:mdd_idx]
    peak_idx = before_mdd["stck_clpr_float"].idxmax()
    peak_date = before_mdd.loc[peak_idx, "date"]
    
    # 회복 찾기 (MDD 이후 고점 회복 시점)
    after_mdd = dd_df.loc[mdd_idx:]
    peak_price = before_mdd.loc[peak_idx, "stck_clpr_float"]
    recovery = after_mdd[after_mdd["stck_clpr_float"] >= peak_price]
    
    recovery_date = recovery["date"].iloc[0] if not recovery.empty else None
    
    return {
        "mdd_peak_date": peak_date.strftime("%Y-%m-%d") if hasattr(peak_date, 'strftime') else str(peak_date),
        "mdd_trough_date": mdd_date.strftime("%Y-%m-%d") if hasattr(mdd_date, 'strftime') else str(mdd_date),
        "mdd_recovery_date": recovery_date.strftime("%Y-%m-%d") if recovery_date and hasattr(recovery_date, 'strftime') else None,
        "mdd_value": float(mdd_value),
        "days_to_trough": (mdd_date - peak_date).days if hasattr(mdd_date - peak_date, 'days') else None,
        "days_to_recovery": (recovery_date - mdd_date).days if recovery_date else None,
    }

---

## Step 6. 전체 리스크 지표 파이프라인

In [11]:
def run_etf_risk_job(
    etf_code: str,
    benchmark_code: str,
    start_date: str,
    end_date: str,
    inception_date: Optional[str] = None,
    *,
    risk_free_rate: float = 0.03,
    chunk_size: int = 90,
) -> Dict[str, Any]:
    """
    ETF + 벤치마크에 대해 퀀트/리스크 지표 계산 및 저장.
    """
    etf_code = str(etf_code).zfill(6)
    benchmark_code = str(benchmark_code).strip()
    
    print(f"\n{'='*60}")
    print(f"ETF 리스크 분석: {etf_code} vs {benchmark_code}")
    print(f"기간: {start_date} ~ {end_date}")
    print(f"{'='*60}")
    
    # 1) 일봉 시계열 수집
    print("\n[1] ETF 일봉 수집...")
    df_etf_raw = fetch_daily_price_range(etf_code, start_date, end_date, chunk_size=chunk_size)
    
    print("\n[2] 벤치마크 일봉 수집...")
    df_bm_raw = fetch_daily_price_range(benchmark_code, start_date, end_date, chunk_size=chunk_size)
    
    # 2) 일간 수익률 추가
    print("\n[3] 수익률 계산...")
    df_etf = enrich_with_returns(df_etf_raw)
    df_bm = enrich_with_returns(df_bm_raw)
    
    # 3) 단일 리스크 지표
    print("\n[4] 리스크 지표 계산...")
    vol_ann = calc_volatility_annual(df_etf)
    mdd = calc_mdd(df_etf)
    sharpe, sortino = calc_sharpe_sortino(df_etf, risk_free_rate=risk_free_rate)
    
    # 4) Beta / Alpha
    beta, alpha = calc_beta_alpha(df_etf, df_bm, risk_free_rate=risk_free_rate)
    
    # 5) 추적오차 및 유동성
    tracking_error = calc_tracking_error(df_etf, df_bm)
    avg_vol_20d = calc_avg_volume(df_etf, days=20)
    
    # 6) 기간별 누적수익률
    period_map = {
        "1M": 21,
        "3M": 63,
        "6M": 126,
        "1Y": 252,
        "3Y": 756,
        "5Y": 1260,
    }
    period_returns = {}
    for label, days in period_map.items():
        period_returns[label] = calc_period_return(df_etf, days)
    period_returns["YTD"] = calc_ytd_return(df_etf)
    
    # 7) 상대수익률
    relative_returns = {}
    for label, days in period_map.items():
        relative_returns[label] = calc_relative_return(df_etf, df_bm, days)
    
    # 8) 장기 MDD
    mdd_3y = calc_mdd_period(df_etf, period_map["3Y"])
    mdd_5y = calc_mdd_period(df_etf, period_map["5Y"])
    mdd_since_inception = None
    if inception_date:
        mdd_since_inception = calc_mdd_since_inception(df_etf, inception_date)
    
    # 9) 드로다운 상세
    dd_stats = calc_drawdown_stats(df_etf)
    dd_period = find_max_drawdown_period(df_etf)
    
    # 10) 요약 row 생성
    as_of_date = df_etf["date"].max().strftime("%Y-%m-%d") if not df_etf.empty else end_date
    
    summary_row = {
        "as_of_date": as_of_date,
        "etf_code": etf_code,
        "benchmark_code": benchmark_code,
        "risk_free_rate": risk_free_rate,
        "inception_date": inception_date,
        
        # 기본 리스크 지표
        "vol_annual": vol_ann,
        "mdd": mdd,
        "sharpe": sharpe,
        "sortino": sortino,
        "beta": beta,
        "alpha": alpha,
        "tracking_error": tracking_error,
        "avg_volume_20d": avg_vol_20d,
        
        # 장기 MDD
        "mdd_3Y": mdd_3y,
        "mdd_5Y": mdd_5y,
        "mdd_since_inception": mdd_since_inception,
    }
    
    # 기간 수익률 추가
    for label, val in period_returns.items():
        summary_row[f"return_{label}"] = val
    
    # 상대수익률 추가
    for label, val in relative_returns.items():
        summary_row[f"relative_return_{label}"] = val
    
    # 드로다운 상세 추가
    for k, v in dd_stats.items():
        summary_row[f"dd_{k}"] = v
    for k, v in dd_period.items():
        summary_row[f"dd_{k}"] = v
    
    df_summary = pd.DataFrame([summary_row])
    
    # 11) 파일 저장
    s_disp = datetime.strptime(start_date, "%Y%m%d").strftime("%Y-%m-%d")
    e_disp = datetime.strptime(end_date, "%Y%m%d").strftime("%Y-%m-%d")
    
    etf_price_path = DAILY_PRICE_DIR / f"{etf_code}_daily_{s_disp}_{e_disp}.csv"
    bm_price_path = DAILY_PRICE_DIR / f"{benchmark_code}_daily_{s_disp}_{e_disp}.csv"
    risk_path = RISK_SUMMARY_DIR / f"{etf_code}_risk_{as_of_date}.csv"
    
    df_etf.to_csv(etf_price_path, index=False)
    df_bm.to_csv(bm_price_path, index=False)
    df_summary.to_csv(risk_path, index=False)
    
    print(f"\n[SAVE] ETF daily    → {etf_price_path}")
    print(f"[SAVE] BM daily     → {bm_price_path}")
    print(f"[SAVE] risk summary → {risk_path}")
    
    return {
        "etf_daily": df_etf,
        "bm_daily": df_bm,
        "risk_summary": df_summary,
        "period_returns": period_returns,
        "relative_returns": relative_returns,
        "drawdown_stats": dd_stats,
        "drawdown_period": dd_period,
    }

---

## 예시 실행: KODEX 200 (최근 2년)

In [12]:
# 토큰 확인
ensure_kis_token()

# 설정
example_etf = "069500"  # KODEX 200
example_benchmark = get_benchmark_code(example_etf)  # KOSPI 200

end_dt = date.today()
start_dt = end_dt - timedelta(days=730)  # 약 2년

start_date = start_dt.strftime("%Y%m%d")
end_date = end_dt.strftime("%Y%m%d")

print(f"Target ETF: {example_etf}")
print(f"Benchmark : {example_benchmark}")
print(f"Period    : {start_date} ~ {end_date}")

# 실행
risk_result = run_etf_risk_job(
    etf_code=example_etf,
    benchmark_code=example_benchmark,
    start_date=start_date,
    end_date=end_date,
    risk_free_rate=0.03,
    chunk_size=90,
)

Target ETF: 069500
Benchmark : 2001
Period    : 20231202 ~ 20251201

ETF 리스크 분석: 069500 vs 2001
기간: 20231202 ~ 20251201

[1] ETF 일봉 수집...
  [fetch] 069500 (STOCK) 20231202 ~ 20240229
  [fetch] 069500 (STOCK) 20240301 ~ 20240529
  [fetch] 069500 (STOCK) 20240530 ~ 20240827
  [fetch] 069500 (STOCK) 20240828 ~ 20241125
  [fetch] 069500 (STOCK) 20241126 ~ 20250223
  [fetch] 069500 (STOCK) 20250224 ~ 20250524
  [fetch] 069500 (STOCK) 20250525 ~ 20250822
  [fetch] 069500 (STOCK) 20250823 ~ 20251120
  [fetch] 069500 (STOCK) 20251121 ~ 20251201

[2] 벤치마크 일봉 수집...
  [fetch] 2001 (INDEX) 20231202 ~ 20240229
  [fetch] 2001 (INDEX) 20240301 ~ 20240529
  [fetch] 2001 (INDEX) 20240530 ~ 20240827
  [fetch] 2001 (INDEX) 20240828 ~ 20241125
  [fetch] 2001 (INDEX) 20241126 ~ 20250223
  [fetch] 2001 (INDEX) 20250224 ~ 20250524
  [fetch] 2001 (INDEX) 20250525 ~ 20250822
  [fetch] 2001 (INDEX) 20250823 ~ 20251120
  [fetch] 2001 (INDEX) 20251121 ~ 20251201

[3] 수익률 계산...

[4] 리스크 지표 계산...

[SAVE] ETF daily 

In [13]:
print("\n" + "="*60)
print("결과 미리보기")
print("="*60)

# ETF 일봉 (최근 5일)
print("\n[ETF 일봉 - 최근 5일]")
display(risk_result["etf_daily"].tail()[["date", "stck_clpr_float", "daily_ret"]])

# 리스크 요약
print("\n[리스크 요약]")
display(risk_result["risk_summary"].T)

# 기간 수익률
print("\n[기간 수익률]")
for k, v in risk_result["period_returns"].items():
    if v is not None:
        print(f"  {k}: {v:.2%}")

# 드로다운 통계
print("\n[드로다운 통계]")
for k, v in risk_result["drawdown_stats"].items():
    if v is not None:
        print(f"  {k}: {v:.4f}")


결과 미리보기

[ETF 일봉 - 최근 5일]


,date,stck_clpr_float,daily_ret
479,2025-11-25,54605.0,0.007286
480,2025-11-26,56080.0,0.027012
481,2025-11-27,56475.0,0.007044
482,2025-11-28,55650.0,-0.014608
483,2025-12-01,55575.0,-0.001348



[리스크 요약]


,0
as_of_date,2025-12-01
etf_code,069500
benchmark_code,2001
risk_free_rate,0.03
inception_date,None
vol_annual,0.214537
mdd,-0.216062
sharpe,1.290588
sortino,1.626318
beta,0.747915



[기간 수익률]
  1M: -4.42%
  3M: 29.13%
  6M: 58.46%
  1Y: 77.11%
  YTD: 77.24%

[드로다운 통계]
  mdd: -0.2161
  avg_drawdown: -0.0679
  drawdown_std: 0.0637
  time_underwater_pct: 0.8450
  current_drawdown: -0.0727


---

## 정리: 11강에서 계산한 지표들

### 7강 연계 - 수익률 지표
- 일간 수익률 (`daily_ret`)
- 기간 수익률 (1M, 3M, 6M, 1Y, 3Y, 5Y)
- YTD 수익률, 설정일 이후 수익률

### 7강 연계 - 리스크 지표
- 연환산 변동성 (`vol_annual`)
- 최대낙폭 (`mdd`, `mdd_3Y`, `mdd_5Y`)
- Sharpe 비율, Sortino 비율

### 7강/8강 연계 - 상대 지표
- Beta, Alpha (CAPM 기반)
- 추적오차 (Tracking Error)
- 상대수익률 (ETF - 벤치마크)

### 7강 연계 - 드로다운 분석
- 드로다운 시계열
- 평균 드로다운, 현재 드로다운
- MDD 발생 기간 (고점→저점→회복)

### 📌 활용 방안
- 10강 메타(상품 특성) + 11강 리스크 지표를 결합하면
- ETF 선택을 위한 종합 스코어링 가능